# Normalisation des sous titres

- transformer le texte en lettre (compliqué...)
- enlever les M. etc en Monsieur

In [1]:
import spacy
import os
import re
import module_traitement as m
spacy.prefer_gpu()
nlp = spacy.load("fr_dep_news_trf")
from spacy.language import Language

In [2]:
# TEST - Pour Matignon - LSF 
file_with_path = m.lister_fichiers_with_path("../data/cr_audio_aligned/")
folder = m.lister_fichiers("../data/cr_audio_aligned/")
output_seg = "test_new_seg_cr/"
output_cleaning = "test_cleaning_cr/"
#output_sent = "../data/sentence_matignon/"

In [4]:
def get_dict_vtt(input):
    with open(input,encoding="utf-8") as f:
        lines = f.readlines()

    dict_sub = {}
    i = 0
    j = 0  

    while j < len(lines): 
        element = lines[j]
        if element.startswith("00:") or element.startswith("01:") or element.startswith("02:"):
            # Extraire le temps de début et de fin
            timing_line = element.strip().split(' --> ')
            start_time, end_time = timing_line

            text = ""
            while j + 1 < len(lines) and not lines[j + 1].startswith("00:") and not lines[j+1].startswith("01:") and not lines[j+1].startswith("02:"):
                j += 1
                content = lines[j]
                text = text + " " + content.strip()

            dict_sub[i] = {'start': start_time, 'end': end_time, 'text': text.strip()}
            i += 1

        j += 1

    return dict_sub

In [24]:
def convertir_grand_nombre(nombre_texte):
    nombre_sans_points = re.sub(r'(\d)\.(\d)', r'\1\2', nombre_texte)
    return nombre_sans_points


In [25]:
def get_dict_vtt_clean(input):
    with open(input,encoding="utf-8") as f:
        lines = f.readlines()

    dict_sub = {}
    i = 0
    j = 0  

    while j < len(lines): 
        element = lines[j]
        if element.startswith("00:") or element.startswith("01:") or element.startswith("02:"):
            # Extraire le temps de début et de fin
            timing_line = element.strip().split(' --> ')
            start_time, end_time = timing_line

            text = ""
            while j + 1 < len(lines) and not lines[j + 1].startswith("00:") and not lines[j+1].startswith("01:") and not lines[j+1].startswith("02:"):
                j += 1
                content = lines[j]
                text = text + " " + content.strip()
                text=text.replace("... -G. Attal : ","")
                text=text.replace("-G. Attal : ","")
                text=text.replace("G. Attal : ","")
                text = text.replace("-Bonjour", "Bonjour")
                text = text.replace("Bonjour.", "Bonjour,")
                text = text.replace(" M."," Monsieur")
                text = text.replace(" Mme"," Madame")
                text = re.sub(r'["“”«»]', '', text)
                if text.startswith("-"):
                    text = text.replace("-","")
                text = convertir_grand_nombre(text)

            dict_sub[i] = {'start': start_time, 'end': end_time, 'text': text.strip()}
            i += 1

        j += 1

    return dict_sub

In [26]:
def segmenter_texte_en_phrases(texte):
    # Utilisation de l'expression régulière pour diviser le texte en phrases
    phrases = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', texte)
    return phrases

In [27]:
pattern = re.compile(r' [A-Z]\.')
for file,name in zip(file_with_path,folder):
    dict_sub = get_dict_vtt_clean(file)
    new_text = ""
    for k,v in dict_sub.items():
        for kk,vv in v.items():
            if kk == "text":
                if vv.startswith("-"):
                    vv = vv.replace("-","")
                new_text = new_text + vv + " "
    phrases = segmenter_texte_en_phrases(new_text)
    with open(f"try/{name}",'w',encoding='utf-8') as f:
        for element in phrases:
            f.write(element + "\n")